In [4]:
import pandas as pd
df = pd.read_csv("./datasets/animal_center_vet.csv")
df.head()

,animal_id,age_upon_outcome,animal_type,breed,age_category
0,A684346,14,Cat,Domestic Shorthair Mix,junior
1,A666430,365,Dog,Beagle Mix,junior
2,A675708,365,Dog,Pit Bull,junior
3,A680386,3285,Dog,Miniature Schnauzer Mix,adult
4,A683115,155,Other,Bat Mix,junior


In [5]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   animal_id         200 non-null    object
 1   age_upon_outcome  200 non-null    int64 
 2   animal_type       200 non-null    object
 3   breed             200 non-null    object
 4   age_category      200 non-null    object
dtypes: int64(1), object(4)
memory usage: 7.9+ KB


,age_upon_outcome
count,200.000000
mean,722.950000
std,1003.411907
min,2.000000
25%,62.000000
50%,365.000000
75%,1095.000000
max,5475.000000


In [11]:
#1.	¿De qué raza (“breed”) es el animal con identificador “A668644”?
df_filtrado = df[df["animal_id"] == "A668644"]
print(df_filtrado["breed"])

df.loc[df["animal_id"] == "A668644", "breed"]

100    Chihuahua Shorthair Mix
Name: breed, dtype: object


100    Chihuahua Shorthair Mix
Name: breed, dtype: object

In [14]:
#2.	¿Cuál es el listado de perros de raza “Pit Bull Mix” y una edad mayor de un año (365 días)?
df_filtrado = df[(df["breed"] == "Pit Bull Mix") & (df["age_upon_outcome"] > 365)]
print(df_filtrado)

    animal_id  age_upon_outcome animal_type         breed age_category
47    A660098               730         Dog  Pit Bull Mix        adult
53    A545127              2190         Dog  Pit Bull Mix        adult
55    A673648              1095         Dog  Pit Bull Mix        adult
113   A552202              3285         Dog  Pit Bull Mix        adult
150   A666941               730         Dog  Pit Bull Mix        adult
165   A687004              1095         Dog  Pit Bull Mix        adult
174   A580685              1460         Dog  Pit Bull Mix        adult
181   A689369              1460         Dog  Pit Bull Mix        adult
184   A625324              1095         Dog  Pit Bull Mix        adult
191   A672082               730         Dog  Pit Bull Mix        adult


In [16]:
#Otra opcion es usar la funcion .query()
df_filtrado = df.query("breed == 'Pit Bull Mix' and age_upon_outcome > 365")
print(df_filtrado)

    animal_id  age_upon_outcome animal_type         breed age_category
47    A660098               730         Dog  Pit Bull Mix        adult
53    A545127              2190         Dog  Pit Bull Mix        adult
55    A673648              1095         Dog  Pit Bull Mix        adult
113   A552202              3285         Dog  Pit Bull Mix        adult
150   A666941               730         Dog  Pit Bull Mix        adult
165   A687004              1095         Dog  Pit Bull Mix        adult
174   A580685              1460         Dog  Pit Bull Mix        adult
181   A689369              1460         Dog  Pit Bull Mix        adult
184   A625324              1095         Dog  Pit Bull Mix        adult
191   A672082               730         Dog  Pit Bull Mix        adult


In [18]:
#3.	¿Cuántos días (“age_upon_outcome”) tiene el cachorro más pequeño de cada especie (“animal_type”)?
gdf = df.groupby("animal_type")["age_upon_outcome"].min()
print(gdf)

animal_type
Cat      14
Dog       2
Other    28
Name: age_upon_outcome, dtype: int64


In [19]:
gdf = pd.pivot_table(df,
                     values="age_upon_outcome",
                     index="animal_type",
                     aggfunc="min")
print(gdf)

             age_upon_outcome
animal_type                  
Cat                        14
Dog                         2
Other                      28


In [ ]:
#4.	Cuatnso dias de media ("age_upon_outcome") tienen los animales de cada especie ("animal_type") separando por tramos de categoría de edad (“age_category”) 
gdf = pd.pivot_table(df, values="age_upon_outcome",
               index="age_category", columns="animal_type",
               aggfunc="mean")
print(gdf)

animal_type           Cat          Dog       Other
age_category                                      
adult         1786.578947  1645.943396  973.333333
junior         120.379310   191.596491  258.700000


In [26]:
gdf = pd.pivot_table(df, values="age_upon_outcome",
               index="animal_type", columns="age_category",
               aggfunc="mean")
print(gdf.T)

animal_type           Cat          Dog       Other
age_category                                      
adult         1786.578947  1645.943396  973.333333
junior         120.379310   191.596491  258.700000


In [ ]:
#Veamos como funciona group by + Unstack
gdf = (df.groupby(["age_category", "animal_type"])["age_upon_outcome"]
         .mean()
         .unstack())
print(gdf)

'''
Opciones
groupby + unstack → más flexible, más bajo nivel.
pivot_table → más directo, más parecido a Excel.
'''

age_category        adult      junior
animal_type                          
Cat           1786.578947  120.379310
Dog           1645.943396  191.596491
Other          973.333333  258.700000


In [36]:
#5.	Mezclando con el segundo fichero, “animal_center_prop.csv”, 
# ¿cuál es la media de edad (“age_upon_outcome”) de cada sexo (“sex_upon_outcome”)?
df_prop = pd.read_csv("./datasets/animal_center_prop.csv")
df_merged = pd.merge(df, df_prop, on="animal_id")
#En la solucion inicial
#merged = df.merge(df_prop, on="animal_in", how="inner")
gdf = df_merged.groupby("sex_upon_outcome")["age_upon_outcome"].mean().sort_values(ascending=False)
print(gdf)


sex_upon_outcome
Neutered Male    1015.463768
Spayed Female     751.859375
Intact Female     588.965517
Unknown           378.350000
Intact Male        97.611111
Name: age_upon_outcome, dtype: float64


In [ ]:
#6.	Escribir en un fichero “report.csv” con el top 10 de animales más pequeños (en edad, “age_upon_outcome”) 
# de color negro (“Black”). Pista para la ordenación: sort_values.
df_black = df_merged[df_merged["color"] == "Black"]
df_min = df_black.sort_values("age_upon_outcome").head(10)
df_min.to_csv("report.csv", index=False, sep=",")

#Por defecto en groupby las filas con NaN en la columna de agrupación se descartan
#Si queremos una categoria aparte
#df.groupby("col", dropna=False)["valor"].mean()

#Esta misma preguta para los pivot_table
'''
pd.pivot_table(df,
               values="age_upon_outcome",
               index="animal_type",
               columns="sex_upon_outcome",
               aggfunc="mean",
               fill_value=0)   # sustituye NaN por 0

pd.pivot_table(df,
               values="age_upon_outcome",
               index="animal_type",
               columns="sex_upon_outcome",
               aggfunc="mean",
               dropna=False)   # incluye NaN como categoría en index/columns
'''